In [100]:
import numpy as np
import pandas as pd
import csv
import scipy.stats as st


## Importing data

In [101]:


# Specify the path to your CSV file
csv_file_path = 'D:\Artificial intelligence\IA 2023 -\Classification\iris\iris.data'

# Initialize an empty array to store the data
data_array = []

# Open the CSV file
with open(csv_file_path, 'r') as csv_file:
    # Create a CSV reader
    csv_reader = csv.reader(csv_file)
    
    # Iterate through each row in the CSV file
    for row in csv_reader:
        converted_row = [float(value) if value.replace('.', '', 1).isdigit() else value for value in row]#loop through each element of the row
        # Append each row to the data array
        data_array.append(converted_row)



df= pd.DataFrame(data_array)
df.head(len(df))



df.rename(columns={0:'sepal length',1:'sepal width',2:'petal length',3:'petal width ',4:'class'},inplace=True)
new_array =data_array
new_array= np.array(new_array)



df.head(len(df))



,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


## Separate data and its label

In [102]:

target=[]
for i in range (len(new_array)):
   if new_array[i][4] == 'Iris-setosa':
      target.append([1,0,0]) 
   elif new_array[i][4] == 'Iris-versicolor':
      target.append([0,1,0])
   else:
      target.append([0,0,1])
target= np.array(target)
new_array = np.delete(new_array, -1, axis=1)
new_array=new_array.astype('f')


In [103]:
new_df = pd.DataFrame(new_array)
corr =  new_df.corr()
print(corr)

          0         1         2         3
0  1.000000 -0.109369  0.871754  0.817954
1 -0.109369  1.000000 -0.420516 -0.356544
2  0.871754 -0.420516  1.000000  0.962757
3  0.817954 -0.356544  0.962757  1.000000


#### Some of features are highly correlated with others, in most of the cases we would've deleted one of them but we don't have much data neither features so the processing will be quick either way
#### Correlation between features and output have been computes and found greater than |0.3|, so there is no reason to delete

## Creating testing data

In [104]:
setosa_std=round(new_df[:50].std(axis="rows"),2)
setosa_mean=round(new_df[:50].mean(axis="rows"),2)

versicolour_std=round(new_df[50:100].std(axis="rows"),2)
versicolour_mean=round(new_df[50:100].mean(axis="rows"),2)
virginica_std=round(new_df[100:150].std(axis="rows"),2)
virginica_mean=round(new_df[100:150].mean(axis="rows"),2)



testing_data1 = np.column_stack([
    np.round(st.norm(loc=mean, scale=std).rvs(size=50),2) for mean, std in zip(setosa_mean, setosa_std)
])
testing_data2 = np.column_stack([
    np.round(st.norm(loc=mean, scale=std).rvs(size=50),2) for mean, std in zip(versicolour_mean, versicolour_std)
])
testing_data3 = np.column_stack([
    np.round(st.norm(loc=mean, scale=std).rvs(size=50),2) for mean, std in zip(virginica_mean, virginica_std)
])

testing_data=np.concatenate((testing_data1,testing_data2,testing_data3))


## Classification class with gradient descent

In [105]:
class Classification ():
    def __init__(self,data,output) -> None:
        self.n=np.shape(data)[1]
        self.m=np.shape(data)[0]
        self.k=3
        self.weights = np.zeros((self.n,self.k))
        self.bias=0
        self.X = data
        self.outputs = output
        self.alpha = 0.1

    def gradient_descent(self,epochs):
        W=self.weights
        b=self.bias
        alpha = self.alpha
 
        for i in range (epochs): 

            z=np.dot(self.X,(W))
            y_hat=self.softmax(z)
            y_hatX=np.dot(np.transpose(self.X),y_hat)
            yX=np.dot(np.transpose(self.X),self.outputs)
            W -= alpha *(y_hatX - yX)
             
        return W

    def softmax(self, z):
        p = np.zeros((self.m, self.k))
        for i in range(self.m):
            max_z = np.max(z[i][:])
            e_z = np.exp(z[i][:] - max_z)
            p[i][:] = e_z / np.sum(e_z)
        return np.array(p)
    
    
    



## Training

In [106]:
data = new_array
array_output = target

cf = Classification(data,array_output)

W=cf.gradient_descent(500)

## Testing

In [107]:
print(np.shape(data))
print(np.shape(testing_data))

res = np.dot(testing_data,(W))

res = cf.softmax(res)


df_res=pd.DataFrame(res)
fails=0
for idx,value in enumerate(res):
    if np.argmax(value)!= np.argmax(target[idx]):
        fails+=1
print(f'Accuracy : {100-fails*100/150}%')
df_res.head(len(res))

(150, 4)
(150, 4)
Accuracy : 86.0%


,0,1,2
0,1.000000e+00,3.983814e-82,0.000000e+00
1,1.000000e+00,5.275438e-74,0.000000e+00
2,1.000000e+00,2.929248e-40,0.000000e+00
3,1.000000e+00,1.419301e-79,0.000000e+00
4,1.000000e+00,9.319064e-97,0.000000e+00
...,...,...,...
145,6.117323e-256,4.506980e-152,1.000000e+00
146,4.430463e-217,2.175043e-112,1.000000e+00
147,2.135450e-226,2.021856e-122,1.000000e+00
148,8.757331e-206,4.666710e-74,1.000000e+00
